# Load Libraries dan Mount Data Google Drive

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#os.listdir('../input/digit-recognizer')
# Any results you write to the current directory are saved as output.

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Load Data

In [0]:
PATH = '/content/drive/My Drive/Colab Notebooks/Tugas 2 Deep Learning/Dataset'

df_train = pd.read_csv(os.path.join(PATH, 'train.csv'))
train_y = df_train['label'].values
train_x = df_train.drop(['label'], axis=1).values


df_test = pd.read_csv(os.path.join(PATH, 'test.csv'))
test_x = df_test.values


print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(42000, 784)
(42000,)
(28000, 784)


# Reshape/Resize Data

In [0]:
IMG_SIZE = 32

In [0]:
# resize
import cv2

def resize(img_array):
    tmp = np.empty((img_array.shape[0], IMG_SIZE, IMG_SIZE))

    for i in range(len(img_array)):
        img = img_array[i].reshape(28, 28).astype('uint8')
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype('float32')/255
        tmp[i] = img
        
    return tmp

train_x_resize = resize(train_x)
test_x_resize = resize(test_x)

In [0]:
train_x_final = np.stack((train_x_resize,)*3, axis=-1)
test_x_final = np.stack((test_x_resize,)*3, axis=-1)
print(train_x_final.shape)
print(test_x_final.shape)

(42000, 32, 32, 3)
(28000, 32, 32, 3)


In [0]:
from keras.utils import to_categorical
train_y_final = to_categorical(train_y, num_classes=10)
print(train_y_final.shape)

Using TensorFlow backend.


(42000, 10)


# VGG19 Model

In [0]:
# models 
from keras.models import Sequential
from keras.applications import VGG19
from keras.layers import Dense, Flatten

vgg19 = VGG19(weights = 'imagenet', 
              include_top = False,
              input_shape=(IMG_SIZE, IMG_SIZE, 3)
              )

model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='sgd', 
              metrics=['accuracy'])

model.summary()





80142336/80134624 [==============================] - 2s 0us/step








Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 20,029,514
Trainable params: 20,029,514
Non-trainable params: 0
_________________________________________________________________


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_x_final, train_y_final, test_size=0.2, random_state=2019)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(33600, 32, 32, 3)
(8400, 32, 32, 3)
(33600, 10)
(8400, 10)


In [0]:
# callback
from keras.callbacks import ModelCheckpoint, EarlyStopping
#es = EarlyStopping(monitor='val_acc', verbose=1, patience=5)
mc = ModelCheckpoint(filepath='mnist-vgg19.h5', verbose=1, monitor='val_acc')
#cb = [es, mc]
cb = [mc]


## **Epoch 100**

In [0]:
history = model.fit(x_train, y_train, 
                    epochs=100, 
                    batch_size=128, 
                    validation_data=(x_test, y_test),
                    callbacks=cb)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 33600 samples, validate on 8400 samples
Epoch 1/100
33600/33600 [==============================] - 27s 791us/step - loss: 0.4929 - acc: 0.8393 - val_loss: 0.0600 - val_acc: 0.9819

Epoch 00001: saving model to mnist-vgg19.h5
Epoch 2/100
33600/33600 [==============================] - 21s 627us/step - loss: 0.0462 - acc: 0.9857 - val_loss: 0.0555 - val_acc: 0.9821

Epoch 00002: saving model to mnist-vgg19.h5
Epoch 3/100
33600/33600 [==============================] - 21s 614us/step - loss: 0.0327 - acc: 0.9902 - val_loss: 0.0322 - val_acc: 0.9899

Epoch 00003: saving model to mnist-vgg19.h5
Epoch 4/100
33600/33600 [==============================] - 20s 608us/step - loss: 0.0248 - acc: 0.9921 - val_loss: 0.0405 - val_acc: 0.9888

Epoch 00004: saving model to mnist-vgg19.h5
Epoch 5/100
33600/33600 [==============================] - 21s 613us/step - loss: 0.0195 - acc: 0.9937 - val_loss: 

In [0]:
preds = model.predict(test_x_final, batch_size=128)

In [0]:
preds.shape

(28000, 10)

In [0]:
results = np.argmax(preds, axis=-1)
results.shape

(28000,)

In [0]:
# submission
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))
sub.head()
df = pd.DataFrame({'ImageId': sub['ImageId'], 'Label': results})
df.to_csv('submission.csv', index=False)
os.listdir('./')

['.config', 'drive', 'mnist-vgg19.h5', 'submission.csv', 'sample_data']

## **Epoch 50**

In [0]:
history = model.fit(x_train, y_train, 
                    epochs=50, 
                    batch_size=128, 
                    validation_data=(x_test, y_test),
                    callbacks=cb)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 33600 samples, validate on 8400 samples
Epoch 1/50
33600/33600 [==============================] - 31s 913us/step - loss: 0.3783 - acc: 0.8827 - val_loss: 0.0890 - val_acc: 0.9726

Epoch 00001: saving model to mnist-vgg19.h5
Epoch 2/50
33600/33600 [==============================] - 21s 620us/step - loss: 0.0425 - acc: 0.9868 - val_loss: 0.0613 - val_acc: 0.9811

Epoch 00002: saving model to mnist-vgg19.h5
Epoch 3/50
33600/33600 [==============================] - 21s 625us/step - loss: 0.0304 - acc: 0.9902 - val_loss: 0.0410 - val_acc: 0.9887

Epoch 00003: saving model to mnist-vgg19.h5
Epoch 4/50
33600/33600 [==============================] - 21s 619us/step - loss: 0.0238 - acc: 0.9925 - val_loss: 0.0360 - val_acc: 0.9896

Epoch 00004: saving model to mnist-vgg19.h5
Epoch 5/50
33600/33600 [==============================] - 21s 614us/step - loss: 0.0178 - acc: 0.9945 - val_loss: 0.023

In [0]:
preds = model.predict(test_x_final, batch_size=128)

In [0]:
preds.shape

(28000, 10)

In [0]:
results = np.argmax(preds, axis=-1)
results.shape

(28000,)

In [0]:
# submission
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))
sub.head()
df = pd.DataFrame({'ImageId': sub['ImageId'], 'Label': results})
df.to_csv('submission.csv', index=False)
os.listdir('./')

['.config', 'drive', 'mnist-vgg19.h5', 'submission.csv', 'sample_data']